In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from pywaffle import Waffle
import math
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import norm
import statistics
from matplotlib.cbook import boxplot_stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from statsmodels.formula.api import ols

In [99]:
#Importar os dados
df = pd.read_csv('base_dummy.csv')
colunas_para_analisar = df.columns[:8]
df = df[colunas_para_analisar]

#Renomeio as colunas pois aparentemente a classe statsmodel nao aceita nomes de variaveis com espaco
df.columns=['Preco','Zona', 'Qualidade_do_Aquecimento',
       'Numero_de_Quartos', 'Area_da_Garagem', 'Area_do_Lote',
       'Numero_de_Banheiros', 'Idade_do_Imovel']

In [100]:
df.head()

,Preco,Zona,Qualidade_do_Aquecimento,Numero_de_Quartos,Area_da_Garagem,Area_do_Lote,Numero_de_Banheiros,Idade_do_Imovel
0,208500.0,Baixa Densidade Residencial,Excelente,3,548.0,8450,2,5
1,181500.0,Baixa Densidade Residencial,Excelente,3,460.0,9600,2,31
2,223500.0,Baixa Densidade Residencial,Excelente,3,608.0,11250,2,7
3,140000.0,Baixa Densidade Residencial,Bom,3,642.0,9550,1,91
4,250000.0,Baixa Densidade Residencial,Excelente,4,836.0,14260,2,8


# ANOVA

1) Determinação da variável em estudo<br/>
Preço da venda<br/>
2) Tipo da variável dependente<br/>
quantitativa contínua<br/>
3) N° de Amostras<br/>
qtd de tipos de zonas (mais de duas amostras)<br/>
4) Relacionamento entre as a mostras<br/>
Independentes<br/>

Hipóteses estatísticas:<br/>
H0 :μSD =μDG=μCD<br/>
H1 : as médias não são todas iguais entre si<br/>

In [102]:
# Função que gera a tabela ANOVA
def gerar_tabela_ANOVA(df,atributo):
    dado = ols('Preco ~ '+atributo,data=df).fit()
    table = sm.stats.anova_lm(dado, typ=2) # DataFrame da ANOVA Tipo 2 
    print(table)

## Utilizando variáveis independentes categóricas

In [103]:
colunas_para_ANOVA = ['Zona', 'Qualidade_do_Aquecimento', 'Numero_de_Quartos', 'Numero_de_Banheiros']
for atributo in colunas_para_ANOVA:
    gerar_tabela_ANOVA(df,atributo)

                sum_sq      df          F        PR(>F)
Zona      6.192316e+11     4.0  50.185133  6.679178e-41
Residual  8.976583e+12  2910.0        NaN           NaN
                                sum_sq      df          F        PR(>F)
Qualidade_do_Aquecimento  9.253751e+11     4.0  77.642508  1.169687e-62
Residual                  8.682560e+12  2914.0        NaN           NaN
                         sum_sq      df           F        PR(>F)
Numero_de_Quartos  5.065400e+11     1.0  162.346218  3.153838e-36
Residual           9.101395e+12  2917.0         NaN           NaN
                           sum_sq      df           F         PR(>F)
Numero_de_Banheiros  1.807298e+12     1.0  675.828074  3.446160e-134
Residual             7.800637e+12  2917.0         NaN            NaN


## Utilizando variáveis independentes continuas

In [104]:
colunas_para_ANOVA = ['Area_da_Garagem', 'Area_do_Lote', 'Idade_do_Imovel']
for atributo in colunas_para_ANOVA:
    gerar_tabela_ANOVA(df,atributo)

                       sum_sq      df           F         PR(>F)
Area_da_Garagem  2.075770e+12     1.0  803.613947  2.376888e-156
Residual         7.532155e+12  2916.0         NaN            NaN
                    sum_sq      df           F        PR(>F)
Area_do_Lote  8.446355e+11     1.0  281.150001  2.574685e-60
Residual      8.763300e+12  2917.0         NaN           NaN
                       sum_sq      df           F        PR(>F)
Idade_do_Imovel  1.267602e+12     1.0  443.338805  9.735612e-92
Residual         8.340334e+12  2917.0         NaN           NaN
